# Logistic Regression

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Data retrieval

In [13]:
# Load the dataset
file_path = '/Users/ceciliaalberti/Documents/INDE_577/datasets/Supply chain logisitcs problem.xlsx'
df = pd.read_excel(file_path)

In [14]:
df

,Order ID,Order Date,Origin Port,Carrier,TPT,Service Level,Ship ahead day count,Ship Late Day count,Customer,Product ID,Plant Code,Destination Port,Unit quantity,Weight
0,1.447296e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,808,14.300000
1,1.447158e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,3188,87.940000
2,1.447139e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2331,61.200000
3,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,847,16.160000
4,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2163,52.340000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9210,1.447305e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,339,2.354118
9211,1.447319e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,339,2.354118
9212,1.447322e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,245,0.294265
9213,1.447145e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683430,PLANT03,PORT09,278,2.480000


# Logistic Regression class 

In [15]:
class GradientDescentLogisticReg:
    def __init__(self) -> None:
        self.X = None
        self.variables = None
        self.y = None
        self.predictor = None
        self.n = None
        self.p = None
        self.bias = None
        self.gamma = None
        self.max_iter = None
        self.eta = None
        self.weights = None
        self.weights_history = []
        self.loss_history = [np.inf]
    
    # Cross entropy loss for one data point
    def cross_entropy_loss(self, y, y_hat):
        return -y * np.log(y_hat) - (1.0 - y) * np.log(1.0 - y_hat)

    # Total cross entropy loss
    def loss(self):
        total_loss = sum(self.cross_entropy_loss(self.y[i], self.sigmoid(x @ self.weights)) 
                         for i, x in enumerate(self.X))
        return total_loss

    # Sigmoid function
    def sigmoid(self, z):
        return 1.0 / (1.0 + np.exp(-z))

    # Gradient of the loss function
    def gradient_L(self):
        sigmoids = np.array([self.sigmoid(x @ self.weights) - self.y[i] for i, x in enumerate(self.X)])
        d_w = sigmoids @ self.X
        return d_w

    # Model fitting with gradient descent
    def fit(self, X, y, bias=True, gamma=0.01, max_iter=100, eta=0.001):
        self.variables = X.columns
        self.predictor = y.name
        
        X = X.to_numpy()
        y = y.to_numpy()
        if bias:
            ones_column = np.ones((X.shape[0], 1))
            X = np.append(ones_column, X, axis=1)
        self.X = X
        self.y = y
        self.n = X.shape[0]
        self.p = X.shape[1]
        self.bias = bias
        self.gamma = gamma
        self.max_iter = max_iter
        self.eta = eta
        
        weights = np.random.rand(self.p)
        self.weights = weights
        self.weights_history.append(weights)
        
        for i in range(1, max_iter + 1):
            dw = self.gradient_L()
            weights = weights - gamma * dw
            self.weights = weights
            self.weights_history.append(weights)
            L = self.loss()
            self.loss_history.append(L)
            if i >= self.max_iter or abs(L - self.loss_history[i - 1]) <= self.eta:
                break
    
    # Predict new data
    def prediction(self, X, weights=None):
        if weights is None:
            weights = self.weights
        
        X = X.to_numpy()
        if self.bias:
            ones_column = np.ones((X.shape[0], 1))
            X = np.append(ones_column, X, axis=1)
        
        labels = np.array([1, 0])
        y_hat = [self.sigmoid(x @ weights) for x in X]
        return [np.random.choice(labels, p=[y_hat_i, 1.0 - y_hat_i]) for y_hat_i in y_hat]

# Data pre-processing

In [16]:

# Create the target column: 1 for delayed shipments, 0 for on-time shipments
df['Delayed'] = df['Ship Late Day count'].apply(lambda x: 1 if x > 0 else 0)

In [23]:
df.isnull().sum()

Order ID                0
Order Date              0
Origin Port             0
Carrier                 0
TPT                     0
Service Level           0
Ship ahead day count    0
Ship Late Day count     0
Customer                0
Product ID              0
Plant Code              0
Destination Port        0
Unit quantity           0
Weight                  0
Delayed                 0
dtype: int64

In [18]:
# Select features and target
features = ['Origin Port', 'Carrier', 'Ship ahead day count', 'Origin Port', 'Plant Code', 'Destination Port']
X = df[features]
y = df['Delayed']

In [19]:
# Encode categorical features for logistic regression
X_encoded = pd.get_dummies(X, drop_first=True)

# Model training

In [20]:
# Train-Test split after undersampling
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Initialize and train your custom Gradient Descent Logistic Regression model
model = GradientDescentLogisticReg()
model.fit(X_train, y_train,  gamma=0.1, max_iter=2000, eta=0.001)

# Make predictions on the test set
predictions = model.prediction(X_test)

/var/folders/74/w4md1nsj02x949w_05548k480000gn/T/ipykernel_49032/3659125927.py:29: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
/var/folders/74/w4md1nsj02x949w_05548k480000gn/T/ipykernel_49032/3659125927.py:19: RuntimeWarning: divide by zero encountered in log
  return -y * np.log(y_hat) - (1.0 - y) * np.log(1.0 - y_hat)
/var/folders/74/w4md1nsj02x949w_05548k480000gn/T/ipykernel_49032/3659125927.py:19: RuntimeWarning: invalid value encountered in scalar multiply
  return -y * np.log(y_hat) - (1.0 - y) * np.log(1.0 - y_hat)


In [21]:
# Compare the actual labels to the predicted ones
conf_matrix = confusion_matrix(y_test, predictions)

print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[1792    0]
 [  51    0]]


The model is highly biased toward predicting all shipments as on time (class 0) because there is a large imbalance in the dataset (9023 on-time vs. 192 delayed shipments).

This can be improved by undersampling the majority class (on-time shipments).